In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected = True)

import cufflinks as cf

import os

# Use Plotly locally
cf.go_offline()

In [2]:
import json
import pandas as pd
import numpy as np
import requests
import zipfile
import io

import plotly.offline as py #working offline
import plotly.graph_objs as go

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [3]:
df_detect = pd.read_csv("/Users/pzhn0857/Documents/evidently+mlflow/electrical_fault/data/detect_dataset.csv")
df_detect.head()

,Output (S),Ia,Ib,Ic,Va,Vb,Vc,Unnamed: 7,Unnamed: 8
0,0,-170.472196,9.219613,161.252583,0.054490,-0.659921,0.605431,NaN,NaN
1,0,-122.235754,6.168667,116.067087,0.102000,-0.628612,0.526202,NaN,NaN
2,0,-90.161474,3.813632,86.347841,0.141026,-0.605277,0.464251,NaN,NaN
3,0,-79.904916,2.398803,77.506112,0.156272,-0.602235,0.445963,NaN,NaN
4,0,-63.885255,0.590667,63.294587,0.180451,-0.591501,0.411050,NaN,NaN


In [4]:
df_detect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12006 entries, 0 to 12005
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Output (S)  12006 non-null  int64  
 1   Ia          12006 non-null  float64
 2   Ib          12006 non-null  float64
 3   Ic          12006 non-null  float64
 4   Va          12006 non-null  float64
 5   Vb          12006 non-null  float64
 6   Vc          12006 non-null  float64
 7   Unnamed: 7  0 non-null      float64
 8   Unnamed: 8  0 non-null      float64
dtypes: float64(8), int64(1)
memory usage: 844.3 KB


In [5]:
df_detect = df_detect.drop(columns=["Unnamed: 7", "Unnamed: 8"])

In [6]:
df_detect["Output (S)"].value_counts()

0    6510
1    5496
Name: Output (S), dtype: int64

In [7]:
fig = px.line(df_detect, x = "Ib", y = "Vb", facet_col="Output (S)", color = "Output (S)")


for annotation in fig.layout.annotations:
    annotation.text = ""

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
    if type(fig.layout[axis]) == go.layout.XAxis:
        fig.layout[axis].title.text = ''
        


fig.update_layout(
    plot_bgcolor = "#ECECEC",
    title = "<b>Current and Voltage in line b</b>",
    title_font_size = 16,
    title_font_color = "black",
    title_pad_t = 5,
    title_pad_l = 20,
    yaxis = dict(
        title_text = "<b> Voltage </b>",
        titlefont = dict(size = 12)
    ),
    
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    )
)

fig.update_yaxes(showticklabels = True, showline = True, linewidth = 2, linecolor = "black")
fig.update_xaxes(showticklabels = True, showline = True, linewidth = 2, linecolor = "black",
                title_text = "<b>Current</b>")



fig.show()

In [8]:
# df_class = pd.read_csv("/Users/pzhn0857/Documents/evidently+mlflow/electrical_fault/data/classData.csv")
# df_class.head()

In [9]:
# df_class['FaultType'] = df_class['G'].astype('str') + df_class['C'].astype('str') + df_class['B'].astype('str') + df_class['A'].astype('str')
# df_class.head()

In [10]:
# df_class.info()

In [11]:
# df_class["FaultType"].unique()

In [12]:
# df_class["FaultType"].value_counts()

In [13]:
# df_no_fault = df_class.loc[df_class["FaultType"] == "0000"]
# df_LL_fault = df_class.loc[df_class["FaultType"] == "0110"]
# df_LG_fault = df_class.loc[df_class["FaultType"] == "1001"]
# df_LLG_fault = df_class.loc[df_class["FaultType"] == "1011"]
# df_LLLG_fault = df_class.loc[df_class["FaultType"] == "1111"]
# df_LLL_fault = df_class.loc[df_class["FaultType"] == "0111"]

In [14]:
features = ['Ia', 'Ib','Ic','Va','Vb','Vc']

detection_data_X = df_detect[features]
detection_data_Y = df_detect['Output (S)']

In [15]:
# class_data_X = df_class[features]
# class_data_Y = df_class['FaultType']

# # Encoding multiclass label
# from sklearn.preprocessing import LabelEncoder 
# le = LabelEncoder()

# class_data_Y = le.fit_transform(class_data_Y)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(detection_data_X,
                                                        detection_data_Y,test_size=0.33,random_state=1)

In [18]:
# X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(class_data_X, class_data_Y, 
#                                                             test_size=0.33,random_state=1)

In [19]:
import argparse

from pprint import pprint

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_recall_fscore_support

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

import mlflow
import mlflow.xgboost

import time

# from utils import fetch_logged_data

In [20]:
def main():
    # enable auto logging
    # this includes xgboost.sklearn estimators
    mlflow.xgboost.autolog()
    mlflow.create_experiment(name='default')

    with mlflow.start_run() as run:

        regressor = xgb.XGBRegressor(n_estimators=20, reg_lambda=1, gamma=0, max_depth=3)
        regressor.fit(X_train_b, y_train_b, eval_set=[(X_test_b, y_test_b)])
        y_pred = regressor.predict(X_test_b)
        mse = mean_squared_error(y_test_b, y_pred)
        run_id = run.info.run_id
        print("Logged data and model in run {}".format(run_id))


if __name__ == "__main__":
    main()

[0]	validation_0-rmse:0.38418
[1]	validation_0-rmse:0.29435
[2]	validation_0-rmse:0.23391
[3]	validation_0-rmse:0.19023
[4]	validation_0-rmse:0.16244
[5]	validation_0-rmse:0.14064
[6]	validation_0-rmse:0.12656
[7]	validation_0-rmse:0.11805
[8]	validation_0-rmse:0.11261
[9]	validation_0-rmse:0.10675
[10]	validation_0-rmse:0.10468
[11]	validation_0-rmse:0.10081
[12]	validation_0-rmse:0.09883
[13]	validation_0-rmse:0.09754
[14]	validation_0-rmse:0.09525
[15]	validation_0-rmse:0.09377
[16]	validation_0-rmse:0.09321
[17]	validation_0-rmse:0.09140
[18]	validation_0-rmse:0.08504
[19]	validation_0-rmse:0.08447
Logged data and model in run 8befa29a80ba430ba07f3fc61fb65382


In [21]:
mlflow_tracking_uri = os.environ.get("MLFLOW_TRACKING_URI", "http://localhost:5000")
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio123'
os.environ['ARTIFACT_ROOT']= 's3://mlflow/'

In [22]:
# def main():
# #     enable auto logging
# #     this includes xgboost.sklearn estimators
# #     mlflow.xgboost.autolog()

# #     with mlflow.start_run() as run:

# #         regressor = xgb.XGBRegressor(n_estimators=20, reg_lambda=1, gamma=0, max_depth=3)
# #         regressor.fit(X_train_b, y_train_b, eval_set=[(X_test_b, y_test_b)])
# #         y_pred_b = regressor.predict(X_test_b)
# #         mse = mean_squared_error(y_test_b, y_pred_b)
# #         run_id = run.info.run_id
# #         print("Logged data and model in run {}".format(run_id))
# #     sleep = False
#     acc = 0
#     loss = 1

#     while acc < 0.9 or loss > 0.1: 
#         # enable auto logging
#         mlflow.xgboost.autolog()
    
#         dtrain = xgb.DMatrix(X_train_b, label=y_train_b)
#         dtest = xgb.DMatrix(X_test_b, label=y_test_b)

        
# #     client = MlflowClient()
# #     n_experiments=0
# #     elif not reuse_experiment:
# #         client = MlflowClient()
# #         n_experiments = len(client.list_experiments())+1
# #         experiment_name = 'experiment_' + str(n_experiments)
# # #     client.delete_experiment(experiment_id=1)
# #         client.create_experiment(name=experiment_name)
#         mlflow.set_tracking_uri(mlflow_tracking_uri)   
#         mlflow.create_experiment('exp', artifact_location='s3://mlflow')
#         mlflow.set_experiment("exp")
#         with mlflow.start_run():
            
    
# #         with mlflow.start_run() as run:

#             # train model
#             params = {
#                 "objective": "multi:softprob",
#                 "num_class": 2,
#                 "learning_rate": 0.3,
#                 "eval_metric": "mlogloss",
#                 "colsample_bytree": 1.0,
#                 "subsample": 1.0,
#                 "seed": 42,
#             }
#             model = xgb.train(params, dtrain, evals=[(dtrain, "train")])

#             # evaluate model
#             y_proba = model.predict(dtest)
#             y_pred = y_proba.argmax(axis=1)
#             loss = log_loss(y_test_b, y_proba)
#             acc = accuracy_score(y_test_b, y_pred)
#             precision, recall, fscore, support = precision_recall_fscore_support(y_test_b, y_pred, average='macro')            
#             mse = mean_squared_error(y_test_b, y_pred)

#             # log metrics
#             mlflow.log_metrics({"log_loss": loss, "accuracy": acc, "precision": precision, 
#                                 "recall": recall, "fscore": fscore, "mse": mse})
#             mlflow.sklearn.log_model(model, "model")
#         mlflow.end_run()
            
# #             i = i+1
# # 
# #     runs = mlflow.search_runs(experiment_ids=experiment_id)
# #     runs.head(10)
            
# #     runs = mlflow.search_runs(experiment_ids=experiment_id,
# #                               order_by=['metrics.mae'], max_results=1)
# #             runs.loc[0]
            
# #             sleep = True
# #             time.sleep(5)
# #             sleep = False

# if __name__ == "__main__":
#     main()

In [23]:
def main():
#     enable auto logging
#     this includes xgboost.sklearn estimators
#     mlflow.xgboost.autolog()

#     with mlflow.start_run() as run:

#         regressor = xgb.XGBRegressor(n_estimators=20, reg_lambda=1, gamma=0, max_depth=3)
#         regressor.fit(X_train_b, y_train_b, eval_set=[(X_test_b, y_test_b)])
#         y_pred_b = regressor.predict(X_test_b)
#         mse = mean_squared_error(y_test_b, y_pred_b)
#         run_id = run.info.run_id
#         print("Logged data and model in run {}".format(run_id))
#     sleep = False
    acc = 0
    loss = 1

    while acc < 0.9 or loss > 0.1: 
        # enable auto logging
        mlflow.xgboost.autolog()
    
        dtrain = xgb.DMatrix(X_train_b, label=y_train_b)
        dtest = xgb.DMatrix(X_test_b, label=y_test_b)

        
#     client = MlflowClient()
#     n_experiments=0
#     elif not reuse_experiment:
#         client = MlflowClient()
#         n_experiments = len(client.list_experiments())+1
#         experiment_name = 'experiment_' + str(n_experiments)
# #     client.delete_experiment(experiment_id=1)
#         client.create_experiment(name=experiment_name)
        mlflow.set_tracking_uri(mlflow_tracking_uri)   
        mlflow.create_experiment('exp', artifact_location='s3://mlflow')
        mlflow.set_experiment("exp")
        with mlflow.start_run():
            
    
#         with mlflow.start_run() as run:

            # train model
            params = {
                'min_child_weight': [1, 5, 10],
                'gamma': [0.5, 1, 1.5, 2, 5],
                'subsample': [0.6, 0.8, 1.0],
                'colsample_bytree': [0.6, 0.8, 1.0],
                'max_depth': [3, 4, 5]
            }
            model = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
#             model = xgb.train(dtrain, evals=[(dtrain, "train")])
            folds = 3
            param_comb = 5

            skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

            random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_b, y_train_b), verbose=3, random_state=1001 )

            random_search.fit(X_train_b, y_train_b)

            # evaluate model
            y_proba = random_search.predict_proba(X_test_b)
            y_pred = y_proba.argmax(axis=1)
            loss = error(y_test_b, y_proba)
            acc = accuracy_score(y_test_b, y_pred)
            precision, recall, fscore, support = precision_recall_fscore_support(y_test_b, y_pred, average='macro')            
            mse = mean_squared_error(y_test_b, y_pred)

            # log metrics
            mlflow.log_metrics({"log_loss": loss, "accuracy": acc, "precision": precision, 
                                "recall": recall, "fscore": fscore, "mse": mse})
            mlflow.sklearn.log_model(model, "model")
            
        mlflow.end_run()
            
#             i = i+1
# 
#     runs = mlflow.search_runs(experiment_ids=experiment_id)
#     runs.head(10)
            
#     runs = mlflow.search_runs(experiment_ids=experiment_id,
#                               order_by=['metrics.mae'], max_results=1)
#             runs.loc[0]
            
#             sleep = True
#             time.sleep(5)
#             sleep = False

if __name__ == "__main__":
    main()

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the

[11:40:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:40:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


NameError: name 'error' is not defined

In [ ]:
#set column mapping for Evidently Profile
data_columns = ColumnMapping()
data_columns.numerical_features = ['Ia', 'Ib','Ic','Va','Vb','Vc']

In [ ]:
#set reference dates
reference_dates = ('2022-01-01 00:00:00','2022-01-28 23:00:00')

#set experiment batches dates
experiment_batches = [
    ('2022-02-01 00:00:00','2022-02-28 23:00:00'),
    ('2022-03-01 00:00:00','2022-03-31 23:00:00'),
    ('2022-04-01 00:00:00','2022-04-30 23:00:00'),
    ('2022-05-01 00:00:00','2022-05-31 23:00:00'),  
    ('2022-06-01 00:00:00','2022-06-30 23:00:00'), 
    ('2022-07-01 00:00:00','2022-07-31 23:00:00'), 
]

In [ ]:
#evaluate data drift with Evidently Profile
def detect_dataset_drift(train_data, test_data, column_mapping, confidence=0.95, threshold=0.5, get_ratio=False):
    """
    Returns True if Data Drift is detected, else returns False.
    If get_ratio is True, returns ration of drifted features.
    The Data Drift detection depends on the confidence level and the threshold.
    For each individual feature Data Drift is detected with the selected confidence (default value is 0.95).
    Data Drift for the dataset is detected if share of the drifted features is above the selected threshold (default value is 0.5).
    """
    
    data_drift_profile = Profile(sections=[DataDriftProfileSection()])
    data_drift_profile.calculate(train_data, test_data, column_mapping=column_mapping)
    report = data_drift_profile.json()
    json_report = json.loads(report)

    drifts = []
    num_features = column_mapping.numerical_features if column_mapping.numerical_features else []
    cat_features = column_mapping.categorical_features if column_mapping.categorical_features else []
    for feature in num_features + cat_features:
        drifts.append(json_report['data_drift']['data']['metrics'][feature]['p_value']) 
        
    n_features = len(drifts)
    n_drifted_features = sum([1 if x<(1. - confidence) else 0 for x in drifts])
    
    if get_ratio:
        return n_drifted_features/n_features
    else:
        return True if n_drifted_features/n_features >= threshold else False



In [ ]:
#evaluate data drift with Evidently Profile
def detect_features_drift(train_data, test_data, column_mapping, confidence=0.95, threshold=0.5, get_pvalues=False):
    """
    Returns 1 if Data Drift is detected, else returns 0. 
    If get_pvalues is True, returns p-value for each feature.
    The Data Drift detection depends on the confidence level and the threshold.
    For each individual feature Data Drift is detected with the selected confidence (default value is 0.95).
    """
    
    data_drift_profile = Profile(sections=[DataDriftProfileSection()])
    data_drift_profile.calculate(train_data, test_data, column_mapping=column_mapping)
    report = data_drift_profile.json()
    json_report = json.loads(report)
    
    drifts = []
    num_features = column_mapping.numerical_features if column_mapping.numerical_features else []
    cat_features = column_mapping.categorical_features if column_mapping.categorical_features else []
    for feature in num_features + cat_features:
        p_value = json_report['data_drift']['data']['metrics'][feature]['p_value']
        if get_pvalues:
            drifts.append((feature, p_value))
        else:
            drifts.append((feature, True if p_value <(1. - confidence) else False))
             
    return drifts

In [ ]:
features_historical_drift = []

for date in experiment_batches:
    drifts = detect_features_drift(X_train_b, 
                           X_test_b, 
                           column_mapping=data_columns, 
                           confidence=0.95,
                           threshold=0.9)
    
    features_historical_drift.append([x[1] for x in drifts])
    
features_historical_drift_frame = pd.DataFrame(features_historical_drift, 
                                               columns = data_columns.numerical_features)

In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = features_historical_drift_frame.astype(int).transpose(),
                   x = [x[1] for x in experiment_batches],
                   y = data_columns.numerical_features,
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0,
                   zmax = 1,
                   showscale = False,
                   colorscale = 'Bluered'
))

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "Feature Drift"
)

fig.show()

In [ ]:
features_historical_drift_pvalues = []

for date in experiment_batches:
    drifts = detect_features_drift(X_train_b, 
                           X_test_b, 
                           column_mapping=data_columns, 
                           confidence=0.95,
                           threshold=0.9,
                           get_pvalues=True)
    
    features_historical_drift_pvalues.append([x[1] for x in drifts])
    
features_historical_drift_pvalues_frame = pd.DataFrame(features_historical_drift_pvalues, 
                                                       columns = data_columns.numerical_features)



In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = features_historical_drift_pvalues_frame.transpose(),
                   x = [x[1] for x in experiment_batches],
                   y = features_historical_drift_pvalues_frame.columns,
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0,
                   zmax = 1,
                   colorscale = 'reds_r'
                   )
               )

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "p-value"
)

fig.show()

Dataset

In [ ]:
dataset_historical_drift = []

for date in experiment_batches:
    dataset_historical_drift.append(detect_dataset_drift(X_train_b, 
                           X_test_b, 
                           column_mapping=data_columns, 
                           confidence=0.95,
                           threshold=0.9))

In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = [[1 if x == True else 0 for x in dataset_historical_drift]],
                   x = [x[1] for x in experiment_batches],
                   y = [''],
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0,
                   zmax = 1,
                   colorscale = 'Bluered',
                   showscale = False
                   )
               )

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "Dataset Drift"
)
fig.show()

In [ ]:
dataset_historical_drift_ratio = []

for date in experiment_batches:
    dataset_historical_drift_ratio.append(detect_dataset_drift(X_train_b, 
                           X_test_b, 
                           column_mapping=data_columns, 
                           confidence=0.95,
                           threshold=0.9,
                           get_ratio=True))

In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = [dataset_historical_drift_ratio],
                   x = [x[1] for x in experiment_batches],
                   y = [''],
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0.5,
                   zmax = 1,
                   colorscale = 'reds'
                  )
               )

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "Dataset Drift"
)
fig.show()

In [ ]:
#log into MLflow
client = MlflowClient()

#set experiment
mlflow.set_experiment('Dataset Drift Analysis with Evidently')

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: 
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Log metrics
        metric = detect_dataset_drift(X_train_b, 
                           X_test_b, 
                           column_mapping=data_columns, 
                           confidence=0.95,
                           threshold=0.9,
                           get_ratio=True)
        
        mlflow.log_metric('dataset drift', metric)

        print(run.info)

In [ ]:
# runs = mlflow.search_runs(experiment_ids=i)
# runs.head(10)
            
# runs = mlflow.search_runs(experiment_ids=experiment_id,
#                           order_by=['metrics.mae'], max_results=1)

In [ ]:
#run MLflow UI (it will be more convinient to run it directly from the terminal)
# !mlflow ui

In [ ]:
#binary classification

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import GaussianNB
# from xgboost import XGBClassifier
# from sklearn import model_selection
# from sklearn.utils import class_weight
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import mean_squared_error
# from sklearn.svm import LinearSVC  
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neural_network import MLPClassifier

In [ ]:
# def run_exps(X_train, X_test, y_train, y_test):    
#     '''
#     Lightweight script to test many models and find winners
#     :param X_train: training split
#     :param y_train: training target vector
#     :param X_test: test split
#     :param y_test: test target vector
#     :return: DataFrame of predictions
#     '''
    
#     dfs = []
    
    
#     models = [
#           ('LogReg', LogisticRegression()), 
#           ('RF', RandomForestClassifier()),
#           ('KNN', KNeighborsClassifier()),
#           ('SVM', SVC()), 
#           ('GNB', GaussianNB()),
#           ('XGB', XGBClassifier())
#         ]
#     results = []
#     names = []
#     scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
#     target_names = ['Fault', 'No Fault']
    
#     classification_report_df = []
#     classification_report_list = []
    
#     y_pred_df = []
#     y_pred_list = []
#     accuracy = []
#     error = []
    
#     for name, model in models:
#         #final = []
#         kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
#         cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
#         clf = model.fit(X_train, y_train)
        
        
#         y_pred = clf.predict(X_test)
#         y_pred_list.append(y_pred)
#         this_y_pred_df = pd.DataFrame(y_pred)
#         this_y_pred_df["model"] = name
#         y_pred_df.append(this_y_pred_df)
        
        
#         results.append(cv_results)
#         names.append(name)
        
        
#         accuracy.append((model.score(X_test,y_test)))
#         error.append(mean_squared_error(y_test,y_pred))
        
#         classification_report_result = classification_report(y_test, y_pred, target_names=target_names,
#                                                             output_dict=True)
#         classification_report_list.append(classification_report_result)
#         this_classification_report_df = pd.DataFrame(classification_report_result).transpose()
#         this_classification_report_df["model"] = name
#         classification_report_df.append(this_classification_report_df)
        
        
#         this_df = pd.DataFrame(cv_results)
#         this_df['model'] = name
#         dfs.append(this_df)
#         final = pd.concat(dfs, ignore_index=True)
    
#     return final, classification_report_df, y_pred_df, accuracy, error

In [ ]:
# binary_final, binary_classification_report_df, y_pred_df, binary_accuracy, binary_error = run_exps(X_train_b, 
#                                                                             X_test_b, y_train_b, y_test_b)

In [ ]:
# binary_classification_report_df_final = pd.concat(binary_classification_report_df, ignore_index=True)

/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.column

[11:40:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:40:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 3/3] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0;, score=1.000 total time=   4.3s
[11:40:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindi

/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.column

[11:40:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:40:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/3] END colsample_bytree=1.0, gamma=5, max_depth=3, min_child_weight=5, subsample=1.0;, score=1.000 total time=   4.3s
[11:40:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindi

/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/pzhn0857/venv/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.column